# Weekly Session 8

## Goal
To develop a basic familiarity and proficiency with numerical root finding.  (The physics is mainly hiding in the lab, but you can see a lot of applications in the awesome exercises.)

## Readings
We will skip the remainder of the linear equations material (6.2) and move directly onto solutions of nonlinear equations.

    6.3.1 Relaxation Method
    
    6.3.2 Convergence Rate
    
    6.3.4 Binary search -- a.k.a. the bisection method

    6.3.5 Newton's method -- a.k.a. the Newton-Raphson method

Although we will not specifically cover extremization in this week's handout, you should also read enough to get a basic idea of what is possible:

    6.4 Maxima and minima of functions
    
## Overview

We will start with a brief introduction to solving non-linear equations, using the relaxation method.  Then you will build up your own module named ```rootfind```, containing functions which implement each of the basic numerical rootfinding methods: the **bisection** method, the **Newton-Raphson** method (for when the derivative is known), and the **Dekker secant** method (for when it is not).  You will also see the value in *hybridizing* these latter two methods with the bisection method.  

As always, scipy has some beautiful versions of these codes available.  So after you have written your own **Newton-Raphson** method, we will switch over to using library functions.

https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.optimize.newton.html

In [ ]:
#For colored cells, currently blue cells, black text 
#https://stackoverflow.com/questions/18024769/adding-custom-styled-paragraphs-in-markdown-cells
from IPython.core.display import HTML
def css_styling():
    styles = open("style.css", "r").read()
    return HTML(styles)
css_styling()

<div class=answer>
This should be a lime green cell with black font if you set the file path correctly</font>
<div/>

## Exercise 0 - Jupyter Notebook Use/Plotting/Coding Tricks

### Keyboard Jupyter Shortcuts
If you haven't noticed, jupyter contains plenty of keyboard shortcuts.  If you'd like to be using keyboard shortcuts (instead of mouse clicks)

Read this and try some out.
http://maxmelnick.com/2016/04/19/python-beginner-tips-and-tricks.html

My favorites are (in sequence) "B," "M," and "Enter" so far, but I'm working on more.

### Some Matplotlib plotting tricks

Run the following code cell.  Notice the last (active) line resets the default font on your plots to a larger size.  You can use similar calls to change many more plotting defaults [link](https://matplotlib.org/api/matplotlib_configuration_api.html).

In [ ]:
import math
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

plt.rcParams['font.size']=14

#line below prints all possible default variables stored in the dictionary rcParams
#plt.rcParams.keys()  

### Passing python function arguments
These semester you have become very experienced passing functions as arguments in other functions.  For instance: 

In [ ]:
def q(x):
    '''Calculates a simple mathematic function 
    
    It is written using numpy's exponential function (not math) so it can operate on numpy arrays.
    accepts: x - float
    returns f(x) - float
    '''
    
    return x

def g(f,x):
    '''Calls another function f(x) and prints its return value 
    
    accepts: 
        x - float
        f(x) - float
    returns 
        None
    '''
    
    print(f(x))
    return None

#call g to act on q at x=2, which prints q(x)=x.  
#Very convoluted way to do something simple, but generalizes to a very powerful tool.
g(q,2)

But what if we want to work with a more complicated function with multiple arguments?  For instance what if I want to call the following: 

In [ ]:
def r(x,y,z):
    return x+y+z

You are probably thinking, this is easy - we can just do the same thing you did with ```x``` above, by passing it to g(f,x) as a separate argument.  But notice that you would have to redefine the function ```g``` to be ```g(f,x,y,z).```  Doesn't this seem inefficient?  

*For instance, what if you want to write ```g``` so it could print the output of any function, not just design it for a single function?*

Lets do that.  The method for doing so is to use a function parameter named ```*args``` which is called "star args".  

In [ ]:
def g1(f,*args):
    '''Calls another function f(x) and prints its return value 
    
    accepts: 
        x - float
        f(x) - float
    returns 
        None
    '''
    
    print("You passed the function (object): ",f, "with arguments:" ,*args)

    print(f(*args))
    return None

So lets run ```g1``` on the function r:

In [ ]:
g1(r,1,2,3)

As if often true, Exercise 0 is a little toolkit I expect you to use/apply later in the notebook.  To get a little practice, play around with the codes above.  For instance, 
* call ```g1``` on another function which takes 2 or 4 arguments (instead of 1 or 3). 
* print individual values of ```*args``` - **see the last link below for this**

<div class=answer>
Answer below.  Add code cells as necessary.
<div/>

In [ ]:
#your code here

Want to know more?  Here are a few links to show you the big picture:
* https://www.python-course.eu/passing_arguments.php
* http://www.informit.com/articles/article.aspx?p=2314818
* https://www.saltycrane.com/blog/2008/01/how-to-use-args-and-kwargs-in-python/

## Exercise 1: Intro to the Relaxation Method and Convergence

Newman uses $f(x) = 2 - e^{-x}$ as his first test case.  Let's start by plotting that and simply checking that the solution in Newman 6.3.1 is approximately correct (an excellent habit).  

In [ ]:
def f(x):
    '''Calculates a simple mathematic function from Newman 6.3.1
    
    It is written using numpy's exponential function (not math) so it can operate on numpy arrays.
    accepts: x - float
    returns f(x) - float
    '''
    
    return 2 - np.exp(-x)

#define the x-range
x = np.linspace(0,3)

#set up the plot
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111)

#plot the function - both left hand side and right hand side
ax.plot(x,f(x),label=r"$2-e^{-x}$")
ax.plot(x,x,label=r"$x$",ls='--')

#add good labels
ax.legend(loc='best')
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.set_aspect('equal')

#add a label to be extra transparent
text1 = ax.annotate(r'solution, $x^* \approx 1.84$', xy=(1.84, 1.84), xytext=(1.8, 1.0),
            arrowprops=dict(facecolor='black', shrink=0.05),
            )

*Notice how I added an arrow and text to the figure above?  This is very useful.*

### Solve the equation $x=2-e^{-x}$ as in Newman 6.3.1 (pg. 251)

In [ ]:
#your code here

Newman does not include a tolerance estimate in this calculation because he hasn't discussed the method yet.  However, you used tolerance exit conditions a lot in Ch. 5 on integrations, so I'm using a term I expect you to know.  *If necessary use the collective brain to remind yourself.*  

Notice that Newman uses a ```for``` loop here?  You can certainly change it to a ```while``` loop, but remember that ```for``` loops are safer than ```while``` loops (and Newman is no fool).

### Improve your algorithm above to measure the tolerance and exit when it is less than $10^{-6}$.

## Exercise 2: Mathematical Relaxation Test 
#### and images/math representations (i.e. more Ex. 0)

In equation 6.75, Newman provides a simple test of whether a function will converge.  It is unfortunate you need the solution to run the test (i.e. is a post-check, not a pre-check).  Thankfully, we have the solution for $x=2-e^{-x}$!

*Note:* I prefer the more algorithmic notation over Newman's $x'$ notation:  $x_{i+1} = f(x_i)$.  It helped me not get mixed up with the meaning of $x'-x^*$ vs. $x-x^*$

### Prove mathematically that the Relaxation Method will work on the function above using Newman's method.  Show your work and briefly explain.

#### Double click this cell for some useful Markdown code.
You can add markdown math here with dollar signs or photograph and add your work.  Simply place your photo in the local directory and then use simple image code (you can remove this image and all of this text of course).  Replace the html image location with your own code.  I've provided a sample. 

![xkcd](https://imgs.xkcd.com/comics/how_it_works.png)

(Note you will need to add any new files you include to the github repo.)

## Exercise 3: Inverting functions

Newman shows the function $x = e^{1-x^2}$ as a bad (or nonconvergent) example and then works out the inversion method.  He leaves it as an exercise to the reader to actually test this method works.  

####  Run the relaxation method on $f'(x)$.    
Do you get Newman's solution (the one he gets by inspection)? 

In [ ]:
#your code here

#### I didn't.  
So what should you do?  Investigate a bit, see if you can "get it" and see if anything "goes wrong."  I'm leaving you to troubleshoot a bit here, but remember to use the collective brain (we got this).

In [ ]:
#your code here

#### Summarize what you learned, particularly about equation 6.75 and how it relates to the "runaway" solution near $x=1$ for this misbehaved little function 

<div class=answer>
<div/>
Answer here (add cell as needed.)

## Exercise 4: Benchmark function for rootfinding
You will use the function
$$
f(x)=\cos x - x
$$

as a benchmark  in developing the following routines.  To orient yourself, first make a plot of the benchmark function. You will want to plot it over a
reasonably wide range, *e.g.*, $-10\leq x \leq 10$, since you will be playing with different choices of initial intervals and guess points in this range.


In [ ]:
#your plot here

#include the following line so you can see where your function crosses zero.
#ax.axhline(y=0,ls='--',color='gray')  
#REMEMBER, OBJECT ORIENTED PLOTS PEOPLE

## Exercise 5 - introduction to bisection AKA binary search (6.3.4)

For efficiency, I have provided a basic code on the bisection method.  Read the code in the local file (with a text editor if necessary).

The bisection method is our baseline method, to which we will be
comparing the other new members.  Moreover, we will be using bisection
steps when we "hybridize" the bisection method with the other, more
sophisticated rootfinding methods.  So, let us start off our new
rootfind methods by bundling the code above into a function.  

Take a close look, and make sure you understand the logic.  The loop structure is designed to provide a better starting point for the approach we will need to take with in the other rootfinding methods.

For instance, all these other methods are based on the idea of finding successive approximations $x_i$ ($i=0$, $1$, $2$, $\ldots$).  Then the "error" is estimated the difference between the two most recent approximate roots, *i.e.*, the TOLERANCE
$$
\epsilon_i = x_i - x_{i-1}.
$$

In [ ]:
import rootfind

 When this estimated error falls below the tolerance, we terminate and return the "last guess" $x_i$.  We can think about the bisection method somewhat the same way (at least, we can phrase the termination condition and return value the same way) if we think of the "midpoint" of our interval as $x_i$.  
 
 **See the usage example below:**

In [ ]:
# test code
if (__name__ == "__main__"):

    # read in external libraries
    import math

    # define function for rootfinding
    def f_bench(x):
        return math.cos(x) - x
    def fp_bench(x):
        return -math.sin(x) - 1

    # bisection tests

    print("bisection(f_bench,(-1,0),1e-10,verbose=True)")
    print(rootfind.bisection(f_bench,(-1,0),1e-10,verbose=True))

    print("bisection(f_bench,(0,1),1e-10,verbose=True)")
    print(rootfind.bisection(f_bench,(0,1),1e-10,verbose=True))

If the tolerance is $10^{-3}$, how many bisections do you expect to need?  What if the tolerance is $10^{-6}$ ?  Or $10^{-9}$ ?  Try this out, and record the number of iteration and the approximate root (or, rather, root-containing interval) you find in each case, for $\cos x - x$.
 
 *Hint:* An incredibly-convenient almost-equality is $2^{10} = 1024 \approx 1000 = 10^3$.  Remember this.  
 Forever.  
 This is why 1024 bytes make a "kilobyte" to a computer scientist.

In [ ]:
#your code here

<div class=answer>
<div/>
Answer here

Try ```bisection``` out on the benchmark function, using the interval $(0,1)$ and a tolerance $10^{-10}$.  How many iterations are required? 

In [ ]:
#your code here

<div class=answer>
<div/>
Answer here

What happens if you ask for a tolerance of $10^{-21}$?  Does it take as many iterations as you expect?  Do you get what you asked for?  Explain roughly what is going on. 

In [ ]:
#your code here

<div class=answer>
<div/>
Answer here

What happens if you use the interval $(-1,0)$, and why?

<div class=answer>
<div/>
Answer here

## Exercise 6: Newton-Raphson method

Now let us go ahead and code a function to implement the Newton-Raphson method.  I have gotten us started by defining the interface.  If you'd like to move this into rootfind.py, so you can call it in Lab 8, feel free to do so.

In [ ]:
def newton(f,fp,x_guess,tolerance,verbose=False,max_iterations=100):
    """ Find root by Newton's method.

    The 'approximation' x_i at each iteration is defined by Newton's
    method, in terms of the previous approximation x_(i-1).
    
    The 'error' x_i-x_(i-1) is defined by the difference in successive
    approximations.

    Returns None if the maximum number of iterations is reached
    without satisfying the tolerance.  Also returns None if
    rootfinding lands on point where f has zero slope.  Otherwise,
    returns final approximation x_i when termination condition is
    reached.

    f: function for rootfinding
    fp: derivative of function for rootfinding (i.e., 'f prime')
    x_guess: initial guess point
    tolerance: error at which search should terminate
    verbose (optional): whether or not to print iteration log
    max_iterations (optional): limit on number of iterations
    """

    # set up for first iteration
    x = x_guess
    error = 2*tolerance  # set error big enough to get past while condition
    iteration_count = 0

    # TODO

    return x


Notice that, in case of complete failure, we should put in a maximum number of iterations as a "safety net", to avoid an infinite loop.

Try your function out on our benchmark function, picking starting points reasonably near the root.  Actually, when it comes time to print out a test case to hand in, please use $x_0=0.5$.

Now, we probably thought of bisection as being pretty "fast" as a way to get to the root.  After all, we saw in **Exercise 3** the error in the bisection method falls exponentially with the number of iterations
$$
\epsilon_i \propto \frac{1}{2^i}.
$$
And "exponential" is "fast", right?  But from Newman you have seen that we expect the Newton-Raphson method to be even faster, at least, once we have gotten close enough to a root that the function is basically described by the first couple of terms in its Taylor series around that root.
 
So let us actually compare the performance of ```bisect``` and ```newton```.  We have already run the necessary test cases.  For ```bisect```, we started with the initial interval $(0,1)$, and, for ```newton```, 
we used $x_0=0.5$, which is the midpoint of this interval.  So it seems reasonable to compare these test cases.  Based on your log of iterations (from the verbose output) make a table indicating how many how many digits of
the answer $x_i$ are correct, with each method, after each iteration ($i=0$, $1$, $2$, $\ldots$).

In [ ]:
#your code here

<div class=answer>
<div/>
Answer here

For the *bisection* method, does the growth in the number of correct digits look linear in the number of iterations, or perhaps quadratic, or exponential?  How about for the *Newton-Raphson* method (the trend here may be
harder to judge by eye)?  Explain these results in terms of the expected convergence rates (from Newman).

<div class=answer>
<div/>
Answer here

What happens if you start Newton's method instead with the initial guess point $x=5$?  How about $x=10$?  Explain what's going on.  It will help to look back at your plot of this function.  

<div class=answer>
<div/>
Answer here

## Exercise 7: More on Newton's method

Here's one more illustration with Newton's method.  Let's try it out on the function

$$
\begin{equation}
f(x)=x^2+1.
\end{equation}
$$

What happens if you pick $x=0$ as the starting point?  How about
$x=0.5$?

In [ ]:
#your code here

<div class=answer>
<div/>
Answer here

*Comment:* Notice that we were applying a rootfinding method, to a function which
had no root. A physics professor of mine promulgated this bit of wisdom:

    **First rule: Think before you try cranking away!**

But, even if we were immune to being so silly as to ever attempt this, there is
a lesson to be learned.  What if our function *had* a root, off somewhere far away from the initial guess point, but the guess point landed us near a local minimum in the function?  Our function would locally look just like this parabola, and the Newton-Raphson method could become trapped oscillating around the minimum, in just this fashion.

**Summary:** Here is an attempt at summarizing the lessons we have learned.  When it works, Newton-Raphson converges much more rapidly than bisection.  However, sometimes it jumps around before converging.  Sometimes it just oscillates, without converging.  Or, if the starting point is not close enough to a root, the method may go
completely astray.

**When it's good, it's very, very good.  When it's bad, it's horrid.**